In [1]:
import torch
from transformers import *
import numpy as np
import sys

sys.path.append("../../src")
from explainer import Archipelago
from application_utils.text_utils import *
from application_utils.text_utils_torch import BertWrapperTorch
from viz.text import interactive_viz_text

%load_ext autoreload
%autoreload 2

## Get Model

In [2]:
device = torch.device("cuda:0")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model_path = "../../downloads/pretrained_bert"
model = BertForSequenceClassification.from_pretrained(model_path)
model_wrapper = BertWrapperTorch(model, device)

## Define Text

In [3]:
text = " It 's predictable , but it jumps through the expected hoops with style and even some depth ."
baseline_token = "_"

## Get Sentiment

In [4]:
text_ids, baseline_ids = get_input_baseline_ids(text, baseline_token, tokenizer)

class_idx = 1
logit = model_wrapper([text_ids])[0,class_idx].item()
polarity = "positive" if logit > 0 else "negative"
print(polarity, logit)

positive 4.163356781005859


## Explain Prediction

In [5]:
xf = TextXformer(text_ids, baseline_ids) 
apgo = Archipelago(model_wrapper, data_xformer=xf, output_indices=class_idx, batch_size=20, interactive=True)
exps, max_magn = apgo.get_interactive_explanations()

## Interactive Visualization

In [6]:
%matplotlib widget
tokens = get_token_list(text_ids, tokenizer)    
interactive_viz_text(exps, tokens, process_stop_words, init_k=3, max_magn=max_magn, fontsize=10)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=3, description='k', max=19), Output()), _dom_classes=('widget-interact',…